In [1]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00


In [2]:
# !pip install --upgrade torch

In [3]:
# =========================== IF YOU WANT TO DOWNGRADE CUDA ON COLAB ===========================

# !wget https://developer.download.nvidia.com/compute/cuda/10.2/Prod/local_installers/cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
# !dpkg -i cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
# !apt-key add /var/cuda-repo-10-2-local-10.2.89-440.33.01/7fa2af80.pub
# !apt-get update
# !apt-get install cuda-10-2
# !export CUDA_PATH=/usr/local/cuda-10.2/
# !nvcc --version

In [4]:
import pandas as pd
import numpy as np
import time
import os
import datetime
import random
import matplotlib.pyplot as plt
from datasets import load_dataset
import torch
from transformers import AutoModel, AutoTokenizer, AdamW, get_linear_schedule_with_warmup, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, SequentialSampler


In [5]:
data = load_dataset('ag_news')
data

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [6]:
df = data['train']
df = pd.DataFrame(df)

In [7]:
df[df['label']==0].head()

,text,label
492,Venezuelans Vote Early in Referendum on Chavez...,0
493,S.Koreans Clash with Police on Iraq Troop Disp...,0
494,Palestinians in Israeli Jails Start Hunger Str...,0
495,Seven Georgian soldiers wounded as South Osset...,0
496,Rwandan Troops Arrive in Darfur (AP) AP - Doze...,0


In [8]:
df = pd.DataFrame(data['train'])
df.head(10)

,text,label
0,Wall St. Bears Claw Back Into the Black (Reute...,2
1,Carlyle Looks Toward Commercial Aerospace (Reu...,2
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,2
3,Iraq Halts Oil Exports from Main Southern Pipe...,2
4,"Oil prices soar to all-time record, posing new...",2
5,"Stocks End Up, But Near Year Lows (Reuters) Re...",2
6,Money Funds Fell in Latest Week (AP) AP - Asse...,2
7,Fed minutes show dissent over inflation (USATO...,2
8,Safety Net (Forbes.com) Forbes.com - After ear...,2
9,Wall St. Bears Claw Back Into the Black NEW Y...,2


In [9]:
df[df['label']==1].head()

,text,label
448,"Phelps, Thorpe Advance in 200 Freestyle (AP) A...",1
449,Reds Knock Padres Out of Wild-Card Lead (AP) A...,1
450,"Dreaming done, NBA stars awaken to harsh Olymp...",1
451,"Indians Beat Twins 7-1, Nearing AL Lead (AP) A...",1
452,"Galaxy, Crew Play to 0-0 Tie (AP) AP - Kevin H...",1


In [10]:
df[df['label']==2].head()

,text,label
0,Wall St. Bears Claw Back Into the Black (Reute...,2
1,Carlyle Looks Toward Commercial Aerospace (Reu...,2
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,2
3,Iraq Halts Oil Exports from Main Southern Pipe...,2
4,"Oil prices soar to all-time record, posing new...",2


In [11]:
df[df['label']==3].head()

,text,label
78,"'Madden,' 'ESPN' Football Score in Different W...",3
79,Group to Propose New High-Speed Wireless Forma...,3
80,AOL to Sell Cheap PCs to Minorities and Senior...,3
81,Companies Approve New High-Capacity Disc Forma...,3
82,Missing June Deals Slow to Return for Software...,3


In [12]:
df['label'].value_counts()

2    30000
3    30000
1    30000
0    30000
Name: label, dtype: int64

In [14]:
num_train_samples = 10000
num_new_samples = int(num_train_samples/df['label'].nunique())

# Group by 'labels' and sample equal number of rows from each group
df = df.groupby('label', group_keys=False).apply(lambda x: x.sample(min(len(x), num_new_samples)))

# Display the resulting DataFrame
df['label'].value_counts()

0    2500
1    2500
2    2500
3    2500
Name: label, dtype: int64

#### We have a perfectly balanced distribution

In [15]:
df.isnull().sum()

text     0
label    0
dtype: int64

###### *Beautiful*

## Labels
### 0 -->  World
### 1 -->  Sports
### 2 -->  Business
### 3 -->  Sci/Tech

## Fine-Tune

In [16]:
!nvidia-smi

Thu Nov 23 04:32:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [18]:
# DEMO
sent = df['text'].iloc[0]
print(f"Original : {sent}")
print(f"Tokenized : {tokenizer.tokenize(sent)}")
print(f"Token Ids: {tokenizer(sent)}")
print(f"Len of token ids : {len(tokenizer(sent)['input_ids'])}")
print(f"Len of attention mask : {len(tokenizer(sent)['attention_mask'])}")

Original : Rights-Sudan: Rhetoric On Darfur Needs to Be Backed By Action &lt;b&gt;...&lt;/b&gt; Human Rights Watch has called on the United Nations Security Council to take substantive action against what it claims are ongoing rights violations in the western Sudanese region of Darfur.
Tokenized : ['rights', '-', 'sudan', ':', 'rhetoric', 'on', 'dar', '##fur', 'needs', 'to', 'be', 'backed', 'by', 'action', '&', 'lt', ';', 'b', '&', 'gt', ';', '.', '.', '.', '&', 'lt', ';', '/', 'b', '&', 'gt', ';', 'human', 'rights', 'watch', 'has', 'called', 'on', 'the', 'united', 'nations', 'security', 'council', 'to', 'take', 'substantive', 'action', 'against', 'what', 'it', 'claims', 'are', 'ongoing', 'rights', 'violations', 'in', 'the', 'western', 'sudanese', 'region', 'of', 'dar', '##fur', '.']
Token Ids: {'input_ids': [101, 2916, 1011, 10411, 1024, 17871, 2006, 18243, 27942, 3791, 2000, 2022, 6153, 2011, 2895, 1004, 8318, 1025, 1038, 1004, 14181, 1025, 1012, 1012, 1012, 1004, 8318, 1025, 1013, 1

In [26]:
start_time = time.time()

max_sent_len = -1
for i in range(num_train_samples):
  max_sent_len = max(max_sent_len, len(tokenizer.encode(df['text'].iloc[i], add_special_tokens=True)))

end_time = time.time()
print(f'Max_length of Input sentences : {max_sent_len}')
print(f'Time Taken : {(end_time - start_time):.2f} seconds')


Max_length of Input sentences : 352
Time Taken : 2.29 seconds


In [27]:
sentences = df.text.values
labels = df.label.values

In [28]:
input_ids = []
attention_masks = []

start_time = time.time()

for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_sent_len,     # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attention masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

end_time = time.time()

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print(f'Total Time Taken : {end_time - start_time} seconds')
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Total Time Taken : 5.125544548034668 seconds
Original:  Rights-Sudan: Rhetoric On Darfur Needs to Be Backed By Action &lt;b&gt;...&lt;/b&gt; Human Rights Watch has called on the United Nations Security Council to take substantive action against what it claims are ongoing rights violations in the western Sudanese region of Darfur.
Token IDs: tensor([  101,  2916,  1011, 10411,  1024, 17871,  2006, 18243, 27942,  3791,
         2000,  2022,  6153,  2011,  2895,  1004,  8318,  1025,  1038,  1004,
        14181,  1025,  1012,  1012,  1012,  1004,  8318,  1025,  1013,  1038,
         1004, 14181,  1025,  2529,  2916,  3422,  2038,  2170,  2006,  1996,
         2142,  3741,  3036,  2473,  2000,  2202, 27737,  2895,  2114,  2054,
         2009,  4447,  2024,  7552,  2916, 13302,  1999,  1996,  2530, 25603,
         2555,  1997, 18243, 27942,  1012,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     

In [29]:
## Train Test Split

dataset = TensorDataset(input_ids, attention_masks, labels)


# Use the 90-10 train-validation Rule
train_size = int(0.9 * len(dataset))
val_size = len(dataset)  - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

9,000 training samples
1,000 validation samples


In [30]:
# Batch size of 16 or 32 is recommended
batch_size = 16

# Here, Training uses Random Sampling.
# DataLoaders are great way for parallelizing, batching and shuffling
train_dataloader = DataLoader(
            train_dataset,  # Train Dataset
            sampler = RandomSampler(train_dataset), # Random Batch Processing
            batch_size = batch_size
        )

# Random batching isn't required for validation
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Sequential Batch Processing
            batch_size = batch_size
        )

In [31]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 4,
    output_attentions = True,   # To Aid in Model Interpretability
    output_hidden_states = True,   # To Aid in Model Interpretability
    use_cache = True  # For more efficient forward pass
    )

model.to('cuda')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [32]:
optimizer = AdamW(model.parameters(),
                  lr = 6e-5, # learning_rate -> default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [33]:
epochs = 4

# Epoch - One forward pass for all training data.
# Note that len(train_dataloader) =  (number of training samples / batch size)
# Total Training Steps (forward passes) = [number of batches] * [number of epochs]
total_steps = len(train_dataloader) * epochs

# Learning Rate Scheduler
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [34]:
# Calculate Accuracy
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [35]:
# Time Formatter
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed))) # round to nearest second
    return str(datetime.timedelta(seconds=elapsed_rounded)) # formats as hh::mm::ss

In [36]:
%env CUDA_LAUNCH_BLOCKING=1
torch.device('cuda')
%env TORCH_USE_CUDA_DSA = 1

env: CUDA_LAUNCH_BLOCKING=1
env: TORCH_USE_CUDA_DSA=1


In [37]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [38]:
device = 'cuda'
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
# torch.cuda.manual_seed_all(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []

# Calculation of Training Time
total_t0 = time.time()


for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    # Time for Each Epoch
    t0 = time.time()
    total_train_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the device using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        output = model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels)
        loss = output.loss
        total_train_loss += loss.item()
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    print("")
    print("Running Validation...")
    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    # Tracking variables
    total_eval_accuracy = 0
    best_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        # No need to construct the compute graph for while Forward propagation.
        # Compute graph required only during BackPropagation
        with torch.no_grad():
            output= model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
        loss = output.loss
        total_eval_loss += loss.item()
        # Move logits and labels to CPU if we are using GPU
        logits = output.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    if avg_val_accuracy > best_eval_accuracy:
        torch.save(model, 'bert_model')
        best_eval_accuracy = avg_val_accuracy
    #print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    #print("  Validation took: {:}".format(validation_time))
    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.38
  Training epcoh took: 0:09:16

Running Validation...
  Accuracy: 0.91

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.19
  Training epcoh took: 0:09:13

Running Validation...
  Accuracy: 0.91

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.11
  Training epcoh took: 0:09:13

Running Validation...
  Accuracy: 0.92

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.06
  Training epcoh took: 0:09:13

Running Validation...
  Accuracy: 0.92

Training complete!
Total training took 0:38:49 (h:mm:ss)


In [39]:
!nvidia-smi

Thu Nov 23 05:12:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    31W /  70W |   9895MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [40]:
total_parameters = sum(p.numel() for p in model.parameters())
print(f'Total Parameters: {total_parameters}')

Total Parameters: 109485316
